In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/comment-category-prediction-challenge/Sample.csv
/kaggle/input/comment-category-prediction-challenge/train.csv
/kaggle/input/comment-category-prediction-challenge/test.csv


In [2]:
import pandas as pd
from sklearn.dummy import DummyClassifier

In [3]:

# Load data
train = pd.read_csv("/kaggle/input/comment-category-prediction-challenge/train.csv")
test = pd.read_csv("/kaggle/input/comment-category-prediction-challenge/test.csv")

In [4]:
# Features and target
X_train = train.drop("label", axis=1)
y_train = train["label"]

In [5]:
# only use numeric columns 
X_train = X_train.select_dtypes(include=["int64", "float64"])
X_test = test.select_dtypes(include=["int64", "float64"])

In [6]:
# Dummy classifier
model = DummyClassifier(strategy="most_frequent")
model.fit(X_train, y_train)

# Predict on TEST data
predictions = model.predict(X_test)

In [7]:
# Load sample submission
submission = pd.read_csv("/kaggle/input/comment-category-prediction-challenge/Sample.csv")

submission["label"] = predictions

submission.to_csv("submission.csv", index=False)

print("Submission created successfully ✅")


Submission created successfully ✅


In [8]:
#Q1 What is the shape of the training dataset?
train.shape


(198000, 15)

In [9]:
#Q2 How many columns are present in the test dataset?

print(test.shape)


(102000, 14)


In [10]:
#Q3 How many columns in the training dataset have object data type?

train.select_dtypes(include=["object"]).shape[1]


5

In [11]:
#Q4 How many numerical columns are present in the training dataset?

train.select_dtypes(include=["int64", "float64"]).shape[1]



9

In [12]:
# Q5 Which of the following columns is of boolean type?

train.select_dtypes(include=["bool"]).columns


Index(['disability'], dtype='object')

In [13]:
# Q6 all columns ka value find k liye

train.isnull().sum()


created_date         0
post_id              0
emoticon_1           0
emoticon_2           0
emoticon_3           0
upvote               0
downvote             0
if_1                 0
if_2                 0
race            145423
religion        145423
gender          145423
disability           0
comment              1
label                0
dtype: int64

In [14]:
#Q7 wo column jo missing value hai
train.isnull().sum()[train.isnull().sum() > 0]


race        145423
religion    145423
gender      145423
comment          1
dtype: int64

In [15]:
#Q8 columns name find ke liye
train.columns[train.isnull().any()]


Index(['race', 'religion', 'gender', 'comment'], dtype='object')

In [16]:
#Q9 Distinct (unique) target classes count karne ke liye ye run karo 👇
train["label"].nunique()


4

In [17]:
#Q10 Total percentage distribution
train["label"].value_counts(normalize=True) * 100


label
0    57.663131
2    31.535354
1     8.039394
3     2.762121
Name: proportion, dtype: float64

In [18]:
#only label 0 ka percantage nikalne k liye
percentage_label_0 = (train["label"] == 0).mean() * 100
print("Percentage of label 0:", percentage_label_0)


Percentage of label 0: 57.66313131313131


In [19]:
#Q11  Median number of upvotes per comment nikalne ke liye ye run karo 👇
train["upvote"].median()


1.0

In [20]:
#Q12  Numerical features me se kis column ka maximum value sabse bada hai, ye nikalne ke liye ye run karo 👇
# Sirf numerical columns
num_cols = train.select_dtypes(include="number")

# Har column ka maximum value
max_values = num_cols.max()

print(max_values)


post_id        129
emoticon_1      47
emoticon_2      11
emoticon_3      17
upvote         201
downvote       107
if_1          1860
if_2          1833
label            3
dtype: int64


In [21]:
#Q13 What is the minimum value of if_2?

train["if_2"].min()


3